In [3]:
import os
import sys
from glob import glob
from tqdm import tqdm
import math
from PIL import Image
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import f1_score
import torch
from torch.nn import functional as F
from torch import nn
from torch.utils.data import DataLoader
sys.path.insert(0, '../')
from utils import load_json, save_pickle
from model import VanillaEfficientNet, load_model
from dataset import TrainDataset, EvalDataset
from transform_settings import configure_transform

In [4]:
from focal_loss.focal_loss import FocalLoss

In [21]:
criterion = FocalLoss(alpha=2, gamma=5)

NameError: name 'long' is not defined

In [12]:
transform = configure_transform('train', 'base')
dataset = TrainDataset('../preprocessed_stratified/train', transform, meta_path='../preprocessed_stratified/metadata.json')
model = VanillaEfficientNet(n_class=18)
loader = DataLoader(dataset, batch_size=64)

Loaded pretrained weights for efficientnet-b3


In [13]:
for imgs, labels in loader:
    break

In [14]:
output = model(imgs)

In [17]:
len(labels)

64

In [20]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable

class FocalLoss(nn.Module):
    def __init__(self, gamma=0, alpha=None, size_average=True):
        super(FocalLoss, self).__init__()
        self.gamma = gamma
        self.alpha = alpha
        if isinstance(alpha, (float, int)): 
            self.alpha = torch.Tensor([alpha,1-alpha])
        if isinstance(alpha,list): self.alpha = torch.Tensor(alpha)
        self.size_average = size_average

    def forward(self, input, target):
        if input.dim()>2:
            input = input.view(input.size(0),input.size(1),-1)  # N,C,H,W => N,C,H*W
            input = input.transpose(1,2)    # N,C,H*W => N,H*W,C
            input = input.contiguous().view(-1,input.size(2))   # N,H*W,C => N*H*W,C
        target = target.view(-1,1)

        logpt = F.log_softmax(input)
        logpt = logpt.gather(1,target)
        logpt = logpt.view(-1)
        pt = Variable(logpt.data.exp())

        if self.alpha is not None:
            if self.alpha.type()!=input.data.type():
                self.alpha = self.alpha.type_as(input.data)
            at = self.alpha.gather(0,target.data.view(-1))
            logpt = logpt * Variable(at)

        loss = -1 * (1-pt)**self.gamma * logpt
        if self.size_average: return loss.mean()
        else: return loss.sum()

In [19]:
output.size()

torch.Size([64, 18])

In [15]:
criterion(output, labels)

RuntimeError: The size of tensor a (64) must match the size of tensor b (18) at non-singleton dimension 1

- CutMix
- Data Imbalance

In [2]:
INFO = '../preprocessed/info.pkl'
META = '../preprocessed/metadata.json'
TASK = 'age'
train_transform = configure_transform('train', 'base')
valid_transform = configure_transform('valid', 'base')
infer_transform = configure_transform('eval', 'base')
metadate = load_json('../preprocessed/metadata.json')

In [3]:
config_age = ('../preprocessed/valid', configure_transform('valid', 'base'), 'age', META)
config_ageg = ('../preprocessed/valid', configure_transform('valid', 'base'), 'ageg', META)

validset_age = TrainDataset(*config_age)
validset_ageg = TrainDataset(*config_ageg)

validloader_age = DataLoader(validset_age, batch_size=32, shuffle=False, drop_last=False)
validloader_ageg = DataLoader(validset_ageg, batch_size=32, shuffle=False, drop_last=False)